In [8]:
# Necessary imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import os
import neptune

In [9]:
# Variables needed for neptune.
proxies = {
 "http": "http://172.16.2.30:8080",
 "https": "http://172.16.2.30:8080",
}

project_name='Hatespeech-CNERG/MH-Bert'
api_token='Api token hidden for security purposes'


In [10]:
# Initialize neptune for logging metrics
neptune.init(project_name,api_token=api_token,proxies=proxies)
neptune.set_project(project_name)

Project(Hatespeech-CNERG/MH-Bert)

In [11]:
# LASER embeddings for each data point are 1024 dimensional, with the ith value stored as ith column in the 
# csv file
cols = [str(i) for i in range(1024)]

In [12]:
# The languages on which we are running the baseline experiment.
langs = ['Arabic','English','German','Indonesian','Italian','Polish','Portugese','Spanish','French']

In [13]:
#Function to run baseline on sampled dataset
# Parameters: 
#   lang - target language to test on. Training data will include points from other languages as well.
#   sample - sample_ratio
#   c - a hyperparameter for the logistic regression model. 
#       Refer to sklearn logistic regression docs (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
#       for explanation of this value
#   samp_type - the way in which data points are sampled from the training data - stratified in our case
#   seed_val - for setting the random seed value for reproducibility.
def train_and_test_allbutone(lang,c,sample,samp_type,seed_val):
    # Define the parameters of the experiment and create a neptune experiment with these parameters
    params = {}
    params['how_train'] = 'all_but_one'
    params['random_seed'] = seed_val
    params['sample_ratio'] = sample
    params['lang'] = lang
    params['samp_strategy'] = samp_type
    params['C'] = c
    name_one = lang+'_'+'all_but_one'+'_'+str(sample)+'_'+samp_type+'_'+str(seed_val)
    neptune.create_experiment(name_one,params=params,send_hardware_metrics=False,run_monitoring_thread=False)
    neptune.append_tag('LR all_but_one')
    neptune.append_tag(lang)
    neptune.append_tag('only_hate')
    neptune.append_tag(samp_type)
    columns = cols+['label']
    # Load training dataset and split the embeddings and labels
    train = pd.DataFrame(columns=columns)
    for l in langs:
        # For other languages, load full dataset
        if l!=lang:
            df = pd.read_csv(os.path.join('../Dataset/embedded/train',l+'.csv'))
            df = df[columns]
            train = pd.concat([train,df],axis=0)
        # For target language, load a sample of the dataset
        elif l==lang:
            df = pd.read_csv(os.path.join('../Dataset/embedded/train',l+'.csv'))
            df = df[columns]
            df_x = df[cols]
            df_y = df['label']
            if samp_type=='equal':
                df_ = train.groupby('label',group_keys=False).apply(lambda x: x.sample(int(sample/2),random_state=seed_val).sample(frac=1,random_state=seed_val).reset_index(drop=True))
                train = pd.concat([train,df_],axis=0)
            elif samp_type=='stratified':
                sss = StratifiedShuffleSplit(n_splits=1,test_size=sample,random_state=seed_val)
                a = list(sss.split(df_x,df_y))
                df_ = df.iloc[a[0][1],:]
                train = pd.concat([train,df_],axis=0)
    # Shuffle the dataset to mix all the language points
    train = train.sample(frac=1,random_state=seed_val).reset_index(drop=True)
    train_x = train[cols]
    train_y = train['label']
    # Initialize a logistic regression model. The class weights are taken as balanced to account for data imbalance
    model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed_val)
    # Train the LR model
    model.fit(train_x,train_y)
    # Read the test and validation datasets
    val = pd.read_csv(os.path.join('../Dataset/embedded/val',lang+'.csv'))
    test = pd.read_csv(os.path.join('../Dataset/embedded/test',lang+'.csv'))
    test_x = test[cols]
    test_y = test['label']
    val_x = val[cols]
    val_y = val['label']
    print(lang,c,sample,samp_type,seed_val)
    print('Training complete!')
    # Get the predictions on the validation and test sets
    pred = model.predict(test_x)
    fscore = f1_score(test_y,pred,average='macro')
    accuracy = accuracy_score(test_y,pred)
    pred_val = model.predict(val_x)
    val_fscore = f1_score(val_y,pred_val,average='macro')
    val_accuracy = accuracy_score(val_y,pred_val)
    print("Macro F1: ",'{0:.6f}'.format(f1_score(test_y,pred,average='macro')))
    print('')
    # Log the accuracy and macro f1 score values to neptune
    neptune.log_metric('val_fscore',val_fscore)
    neptune.log_metric('val_acc',val_accuracy)
    neptune.log_metric('test_fscore',fscore)
    neptune.log_metric('test_accuracy',accuracy)
    neptune.stop()
    return fscore

In [14]:
# Run the above function for all languages, and 4 chosen c values, for 5 seed values each.
for lang in langs:
    for c in [0.01,0.1,1,10]:
        for samp in [16,32,64,128,256]:
            for ty in ['stratified']:
                for seeds in [2018,2019,2020,2021,2022]:
                    seed_val = seeds
                    np.random.seed(seed_val)
                    train_and_test_allbutone(lang,c,samp,ty,seed_val)

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8029
Arabic 0.01 16 stratified 2018
Training complete!
Macro F1:  0.296838

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8033
Arabic 0.01 16 stratified 2019
Training complete!
Macro F1:  0.298860

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8037
Arabic 0.01 16 stratified 2020
Training complete!
Macro F1:  0.297850

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8041
Arabic 0.01 16 stratified 2021
Training complete!
Macro F1:  0.298860

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8043
Arabic 0.01 16 stratified 2022
Training complete!
Macro F1:  0.297850

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8045
Arabic 0.01 32 stratified 2018
Training complete!
Macro F1:  0.298860

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8048
Arabic 0.01 32 stratified 2019
Training complete!
Macro F1:  0.298860

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8050
Arabic 0.01 32 stratified 2020
Training

/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 1 16 stratified 2018
Training complete!
Macro F1:  0.451482

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8178


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 1 16 stratified 2019
Training complete!
Macro F1:  0.449832

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-8181


...

French 10 256 stratified 2020
Training complete!
Macro F1:  0.420982

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9533


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


French 10 256 stratified 2021
Training complete!
Macro F1:  0.420982

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9534


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


French 10 256 stratified 2022
Training complete!
Macro F1:  0.417423



In [16]:
# Function to run zero shot experiment i.e. without any data from the target language. 
def train_and_test_zeroshot(lang,c,sample,seed_val):
    # Define the parameters of the experiment and create a neptune experiment with these parameters
    params = {}
    params['how_train'] = 'all_but_one'
    params['random_seed'] = seed_val
    params['sample_ratio'] = sample
    params['lang'] = lang
    params['C'] = c
    name_one = lang+'_'+'all_but_one'+'_'+str(sample)+'_'+str(seed_val)
    neptune.create_experiment(name_one,params=params,send_hardware_metrics=False,run_monitoring_thread=False)
    neptune.append_tag('LR all_but_one')
    neptune.append_tag(lang)
    neptune.append_tag('only_hate')
    neptune.append_tag('zero shot')
    train = pd.DataFrame()
    columns = cols+['label']
    # Load training dataset and split the embeddings and labels. Only other language points are loaded. 
    for l in langs:
        if l!=lang:
            df = pd.read_csv(os.path.join('../Dataset/embedded/train',l+'.csv'))
            df = df[columns]
            train = pd.concat([train,df],axis=0)
    # Shuffle the dataset
    train = train.sample(frac=1,random_state=seed_val).reset_index(drop=True)
    train_x = train[cols]
    train_y = train['label']
    # Initialize a logistic regression model. The class weights are taken as balanced to account for data imbalance
    model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed_val)
    # Train the model
    model.fit(train_x,train_y)
    # Load the validation and test datasets
    val = pd.read_csv(os.path.join('../Dataset/embedded/val',lang+'.csv'))
    test = pd.read_csv(os.path.join('../Dataset/embedded/test',lang+'.csv'))
    test_x = test[cols]
    test_y = test['label']
    val_x = val[cols]
    val_y = val['label']
    print(lang,c,seed_val)
    print('Training complete!')
    # Get the predictions on the validation and test sets
    pred = model.predict(test_x)
    fscore = f1_score(test_y,pred,average='macro')
    accuracy = accuracy_score(test_y,pred)
    pred_val = model.predict(val_x)
    val_fscore = f1_score(val_y,pred_val,average='macro')
    val_accuracy = accuracy_score(val_y,pred_val)
    print("Macro F1: ",'{0:.6f}'.format(f1_score(test_y,pred,average='macro')))
    print('')
    # Log the accuracy and f1 scores to neptune
    neptune.log_metric('val_fscore',val_fscore)
    neptune.log_metric('val_acc',val_accuracy)
    neptune.log_metric('test_fscore',fscore)
    neptune.log_metric('test_accuracy',accuracy)
    neptune.stop()
    return fscore

In [17]:
for lang in langs:
    for c in [0.01,0.1,1,10]:
        for seeds in [2018,2019,2020,2021,2022]:
            seed_val = seeds
            np.random.seed(seed_val)
            train_and_test_zeroshot(lang,c,0,seed_val)

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9537
Arabic 0.01 2018
Training complete!
Macro F1:  0.296838

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9538
Arabic 0.01 2019
Training complete!
Macro F1:  0.296838

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9539
Arabic 0.01 2020
Training complete!
Macro F1:  0.296838

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9540
Arabic 0.01 2021
Training complete!
Macro F1:  0.296838

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9541
Arabic 0.01 2022
Training complete!
Macro F1:  0.296838

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9542
Arabic 0.1 2018
Training complete!
Macro F1:  0.412779

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9543
Arabic 0.1 2019
Training complete!
Macro F1:  0.412779

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9545
Arabic 0.1 2020
Training complete!
Macro F1:  0.412779

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9547
Arabic 0.1 2021
Training 

/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 1 2018
Training complete!
Macro F1:  0.449832

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9550


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 1 2019
Training complete!
Macro F1:  0.449832

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9551


...

French 10 2020
Training complete!
Macro F1:  0.424528

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9746


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


French 10 2021
Training complete!
Macro F1:  0.419714

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-9748


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


French 10 2022
Training complete!
Macro F1:  0.428063

